# Modelo de Detección de Fraude en Tarjetas de Crédito

Este notebook desarrolla un **modelo predictivo** para identificar transacciones fraudulentas, diseñado para ejecutarse en la **edición Community de Databricks**. Integra **MLflow** para el seguimiento de experimentos y aborda desafíos como el **desbalance de clases** y la evaluación eficiente del modelo, almacenando los resultados y modelos localmente debido a las limitaciones de la plataforma.


---

## Configuraciones

### Datos de Entrada
- El conjunto de datos consiste en transacciones preprocesadas y enriquecidas con características ingenierizadas.
- **Formato de origen**: Tabla Delta.
- **Ruta**: `/FileStore/tables/output_delta_table_datapipe_feature_eng_to_train`.

### Datos de Salida
- **Ruta de guardado del modelo**: `/dbfs/tmp/fraud_detection_cv_model`.
  - El modelo entrenado se guarda localmente para su uso en procesos posteriores.
- **Seguimiento de experimentos**: `/Users/tu-correo@example.com/fraud_detection_experiment`.
  - Todos los parámetros, métricas y artefactos se rastrean mediante MLflow.

### Dependencias
- **MLflow**: Utilizado para el seguimiento de experimentos y almacenamiento de metadatos del modelo.
- **Spark**: Para el manejo escalable de datos e ingeniería de características.
- **Delta Lake**: Asegura un almacenamiento y recuperación de datos eficiente.

---

## Alcance

Este notebook abarca la **fase de entrenamiento y evaluación del modelo** dentro del pipeline de detección de fraudes. Se asume que la preprocesamiento de datos y la ingeniería de características se han realizado previamente en un pipeline separado.


In [0]:
!pip install mlflow
!pip install pytest

  Using cached mlflow-2.19.0-py3-none-any.whl (27.4 MB)
  Using cached SQLAlchemy-2.0.37-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl (114 kB)
  Using cached docker-7.1.0-py3-none-any.whl (147 kB)
  Using cached flask-3.1.0-py3-none-any.whl (102 kB)
  Using cached mlflow_skinny-2.19.0-py3-none-any.whl (5.9 MB)
  Using cached gunicorn-23.0.0-py3-none-any.whl (85 kB)
  Using cached Markdown-3.7-py3-none-any.whl (106 kB)
  Using cached alembic-1.14.1-py3-none-any.whl (233 kB)
  Using cached sqlparse-0.5.3-py3-none-any.whl (44 kB)
  Using cached opentelemetry_api-1.29.0-py3-none-any.whl (64 kB)
  Using cached opentelemetry_sdk-1.29.0-py3-none-any.whl (118 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl (20 kB)
  Using cached importlib_metadata-8.6.1-py3-none-any.whl (26 kB)
  Using cached PyYAML-6.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (737 kB)
  Using cached GitPython-3.1.44-py3-none-any.wh

## Just for explore the dataset


In [0]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder \
    .appName("Read Delta Table") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Ruta del archivo Delta
delta_path = "/FileStore/tables/output_delta_table_datapipe_feature_eng_to_train"

# Leer el archivo Delta en un DataFrame
df = spark.read.format("delta").load(delta_path)

# Mostrar las primeras filas del DataFrame
df.show(20, truncate=False)


+--------------+-------------+-----------------------+------------------+-----------------+----------------+------------+--------+-----------------+--------------------+------------------+-----------------------+-----------+-----------+
|transaction_id|customer_id  |timestamp              |amount            |merchant_category|merchant_country|card_present|is_fraud|timestamp_seconds|transaction_velocity|amount_velocity   |merchant_category_count|hour_of_day|day_of_week|
+--------------+-------------+-----------------------+------------------+-----------------+----------------+------------+--------+-----------------+--------------------+------------------+-----------------------+-----------+-----------+
|17286088af61  |CUST_00000001|2024-09-21 17:06:23.596|37.07460298352177 |entertainment    |US              |true        |false   |1726938383       |3                   |351.3893987739623 |5                      |17         |7          |
|cac80a102495  |CUST_00000001|2024-10-31 13:20:01.20

In [0]:
df.describe().show()

+-------+--------------+-------------+------------------+-----------------+----------------+--------------------+--------------------+------------------+-----------------------+------------------+------------------+
|summary|transaction_id|  customer_id|            amount|merchant_category|merchant_country|   timestamp_seconds|transaction_velocity|   amount_velocity|merchant_category_count|       hour_of_day|       day_of_week|
+-------+--------------+-------------+------------------+-----------------+----------------+--------------------+--------------------+------------------+-----------------------+------------------+------------------+
|  count|        800218|       800218|            800218|           800218|          800218|              800218|              800218|            800218|                 800218|            800218|            800218|
|   mean|      Infinity|         null|123.25789083184193|             null|            null|1.7283998196111722E9|  1.6092014925932683|19

# 1. Configuración Inicial: Logging y Sesión de Spark

In [0]:
import logging
from pyspark.sql import SparkSession

# Configuración del logger
def setup_logging():
    logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
    return logging.getLogger(__name__)

logger = setup_logging()

# Crear una sesión de Spark
def create_spark_session(app_name="Training Pipeline"):
    try:
        spark = SparkSession.builder \
            .appName(app_name) \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
            .getOrCreate()
        logger.info("Spark session created successfully.")
        return spark
    except Exception as e:
        logger.error("Failed to create Spark session: %s", str(e))
        raise


## Load and Prepare Data

### **Propósito:**
Carga datos desde un archivo en formato Delta y prepara las características para el entrenamiento del modelo de detección de fraudes.

### **Proceso:**
1. **Cargar datos:**
   - Lee el archivo Delta ubicado en la ruta especificada (`delta_path`).
   - Verifica que los datos se hayan cargado correctamente.
2. **Ensambles de características:**
   - Usa `VectorAssembler` para combinar las columnas indicadas en `feature_columns` en una sola columna llamada `features`.
3. **Conversión de la etiqueta:**
   - Convierte la columna de etiqueta (`label_column`) a tipo entero para que sea compatible con el modelo.

### **Output:**
Un DataFrame que incluye:
- **`features`**: Columna con las características combinadas para el modelo.
- **`label_column`**: La columna de etiqueta convertida a tipo entero.


In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col

# Cargar datos desde Delta y ensamblar características
def load_and_prepare_data(spark, delta_path, feature_columns, label_column):
    try:
        # Cargar los datos desde Delta
        df = spark.read.format("delta").load(delta_path)
        logger.info("Data loaded successfully from Delta.")
        
        # Crear columna 'features'
        assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
        df = assembler.transform(df)
        logger.info("Features assembled successfully.")
        
        # Convertir la columna de etiqueta a entero
        df = df.withColumn(label_column, col(label_column).cast("integer"))
        return df
    except Exception as e:
        logger.error("Error loading and preparing data: %s", str(e))
        raise

## Add Class Weight

### **Propósito:**
Aborda el problema de desbalance de clases en el conjunto de datos asignando pesos a cada registro, de manera que las clases minoritarias (fraudes) tengan mayor influencia en el modelo durante el entrenamiento.

### **Proceso:**
1. **Cálculo de totales:**
   - Calcula el número total de registros, transacciones fraudulentas y no fraudulentas en el conjunto de datos.
2. **Determinación de pesos:**
   - Calcula un peso para cada clase:
     - `fraud_weight`: Peso asignado a las transacciones fraudulentas.
     - `non_fraud_weight`: Peso asignado a las transacciones no fraudulentas.
3. **Asignación de pesos:**
   - Crea una nueva columna `class_weight` que contiene los pesos correspondientes:
     - `fraud_weight` para registros etiquetados como fraude.
     - `non_fraud_weight` para registros etiquetados como no fraude.

### **Output:**
Un DataFrame que incluye:
- **`class_weight`**: Columna con los pesos asignados a cada registro para balancear las clases durante el entrenamiento.


In [0]:
from pyspark.sql.functions import when

# Agregar la columna 'class_weight'
def add_class_weight(df, label_column="is_fraud"):
    try:
        total_count = df.count()
        fraud_count = df.filter(col(label_column) == 1).count()
        non_fraud_count = df.filter(col(label_column) == 0).count()

        fraud_weight = total_count / (fraud_count * 2)
        non_fraud_weight = total_count / (non_fraud_count * 2)

        df = df.withColumn(
            "class_weight",
            when(col(label_column) == 1, fraud_weight).otherwise(non_fraud_weight)
        )
        logger.info("Class weights added successfully.")
        return df
    except Exception as e:
        logger.error("Error adding class weights: %s", str(e))
        raise


## Train Model with Cross-Validation and MLflow

### **Propósito:**
Entrenar un modelo de regresión logística para detectar fraudes en transacciones de tarjetas de crédito, utilizando validación cruzada para optimizar hiperparámetros y rastrear los experimentos en **MLflow**.

### **Proceso:**
1. **Configuración de MLflow:**
   - Establece el experimento de MLflow para rastrear los parámetros, métricas y modelos.

2. **Definición del modelo:**
   - Configura un modelo de regresión logística con las siguientes columnas:
     - **`features`**: Características del conjunto de datos.
     - **`is_fraud`**: Etiqueta de clase.
     - **`class_weight`**: Pesos para balancear las clases.

3. **Cuadrícula de hiperparámetros:**
   - Define una cuadrícula para el parámetro `regParam` con valores `[0.01, 0.1, 1.0]`.

4. **Validación cruzada:**
   - Configura un esquema de validación cruzada con 3 particiones para evaluar las combinaciones de hiperparámetros.
   - Usa el **Área Bajo la Curva (AUC)** como métrica de evaluación.

5. **Entrenamiento del modelo:**
   - Ajusta el modelo utilizando validación cruzada y selecciona el mejor modelo según la métrica de AUC.

6. **Registro en MLflow:**
   - Registra los parámetros y el modelo entrenado en MLflow.

### **Output:**
El modelo de regresión logística entrenado con los hiperparámetros óptimos:
- **`bestModel`**: El mejor modelo seleccionado basado en AUC.
- **Rastreo en MLflow**: Incluye parámetros, métricas y artefactos del modelo.


In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import mlflow
import mlflow.spark

# Entrenar el modelo con validación cruzada y MLflow
def train_model_with_cv(df, experiment_name):
    try:
        # Configurar MLflow
        mlflow.set_experiment(experiment_name)

        with mlflow.start_run() as run:
            # Configurar el modelo
            lr = LogisticRegression(featuresCol="features", labelCol="is_fraud", weightCol="class_weight", maxIter=10)

            # Configurar la cuadrícula de hiperparámetros
            paramGrid = ParamGridBuilder() \
                .addGrid(lr.regParam, [0.01, 0.1, 1.0]) \
                .build()

            # Configurar la validación cruzada
            evaluator = BinaryClassificationEvaluator(labelCol="is_fraud", metricName="areaUnderROC")
            cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=3)

            # Entrenar el modelo
            logger.info("Training the model with cross-validation.")
            cvModel = cv.fit(df)

            # Registrar el modelo en MLflow
            mlflow.log_param("RegParam", [0.01, 0.1, 1.0])
            mlflow.spark.log_model(cvModel.bestModel, "fraud_detection_model")
            logger.info("Model logged to MLflow.")
        
        return cvModel.bestModel
    except Exception as e:
        logger.error("Error during model training: %s", str(e))
        raise

## Evaluar el Modelo

### **Propósito:**
Evaluar el rendimiento del modelo entrenado para detectar fraudes utilizando la métrica **AUC** (Área Bajo la Curva ROC).

### **Proceso:**
1. **Generar Predicciones:**
   - El modelo entrenado se aplica al conjunto de datos de entrada para generar una columna de predicciones.
   
2. **Configurar el Evaluador:**
   - Se utiliza el evaluador `BinaryClassificationEvaluator` para calcular la métrica de **AUC** (Área Bajo la Curva ROC).
   
3. **Calcular el AUC:**
   - El evaluador calcula el AUC en función de las predicciones generadas por el modelo.
   
4. **Registrar el Resultado:**
   - El valor de AUC se registra en los logs para análisis posterior.

### **Output:**
- **AUC**: Un valor numérico que representa la calidad del modelo en la detección de fraudes, donde un valor cercano a 1 indica un excelente rendimiento.


In [0]:
# Evaluar el modelo
def evaluate_model(model, df):
    try:
        # Generar predicciones
        predictions = model.transform(df)

        # Evaluar el modelo
        evaluator = BinaryClassificationEvaluator(labelCol="is_fraud", metricName="areaUnderROC")
        auc = evaluator.evaluate(predictions)
        logger.info(f"Model evaluation completed. AUC: {auc:.4f}")
        return auc
    except Exception as e:
        logger.error("Error during model evaluation: %s", str(e))
        raise


# 6. Ejecución Principal 

- Configura y ejecuta todo el pipeline de detección de fraudes.
- Carga los datos, agrega pesos para balancear clases, entrena el modelo y evalúa su rendimiento.
- Usa MLflow para rastrear los experimentos y resultados.

In [0]:
if __name__ == "__main__":
    try:
        # Configuración inicial
        logger.info("Starting the fraud detection pipeline.")
        spark = create_spark_session()

        # Rutas y configuración
        delta_path = "/FileStore/tables/output_delta_table_datapipe_feature_eng_to_train"
        model_path = "/dbfs/tmp/fraud_detection_cv_model"
        experiment_name = "/Users/miguel.program.73@gmail.com/fraud_detection_experiment"
        feature_columns = [
            "transaction_velocity",
            "amount_velocity",
            "merchant_category_count",
            "hour_of_day",
            "day_of_week"
        ]
        label_column = "is_fraud"

        # Cargar y preparar los datos
        data = load_and_prepare_data(spark, delta_path, feature_columns, label_column)

        # Agregar la columna 'class_weight'
        data_with_weights = add_class_weight(data, label_column)

        # Entrenar el modelo
        best_model = train_model_with_cv(data_with_weights, experiment_name)

        # Evaluar el modelo
        auc = evaluate_model(best_model, data_with_weights)
        logger.info(f"Pipeline completed successfully. AUC: {auc:.4f}")
        print(f"Model evaluation completed. AUC: {auc:.4f}")

    except Exception as e:
        logger.error("Pipeline execution failed: %s", str(e))
        raise


2025/01/21 22:58:11 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().
2025/01/21 22:58:32 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: dbfs:/databricks/mlflow-tracking/2662644693673107/1913238ff081446fafc46bc1655e2741/artifacts/fraud_detection_model/sparkml, flavor: spark). Fall back to return ['pyspark==3.3.2', 'pandas<2']. Set logging level to DEBUG to see the full traceback. 


🏃 View run puzzled-duck-40 at: https://community.cloud.databricks.com/ml/experiments/2662644693673107/runs/1913238ff081446fafc46bc1655e2741
🧪 View experiment at: https://community.cloud.databricks.com/ml/experiments/2662644693673107
Model evaluation completed. AUC: 0.9911


## Descripción de las Pruebas

### `test_load_and_prepare_data`
- Verifica que la función cargue los datos desde Delta correctamente.
- Asegura que las columnas `features` y `is_fraud` estén presentes en el DataFrame.

### `test_add_class_weight`
- Valida que se genere la columna `class_weight` correctamente.
- Verifica que los pesos sean distintos para las clases de fraude y no fraude.

### `test_train_model_with_cv`
- Comprueba que la función entrene un modelo de regresión logística utilizando validación cruzada.
- Valida que el modelo resultante sea del tipo `LogisticRegressionModel`.

### `test_evaluate_model`
- Valida que el modelo pueda generar predicciones.
- Verifica que el AUC calculado esté dentro del rango válido de 0 a 1.

**Nota:** Estas pruebas unitarias, debido a limitaciones de tiempo y a la imposibilidad de ejecutarlas directamente en el mismo notebook, se dejaron de forma hipotética (no las ejecuté como tal, pero aún asi las hice por el requerimiento). Según la metodología de pytest, estas pruebas se ejecutan en archivos `.py`. Por esta razón, decidí priorizar otros pasos del proceso.


In [0]:
import pytest

@pytest.fixture(scope="session")
def spark():
    """Crea una sesión de Spark para las pruebas."""
    return SparkSession.builder \
        .appName("TestFraudDetection") \
        .master("local[*]") \
        .getOrCreate()

@pytest.fixture
def sample_data(spark):
    """Crea un DataFrame de prueba."""
    data = [
        ("T1", "C1", 3, 150.0, 5, 12, 3, 0),
        ("T2", "C1", 1, 100.0, 4, 18, 2, 1),
        ("T3", "C2", 2, 200.0, 3, 14, 4, 0)
    ]
    schema = ["transaction_id", "customer_id", "transaction_velocity", "amount_velocity",
              "merchant_category_count", "hour_of_day", "day_of_week", "is_fraud"]
    return spark.createDataFrame(data, schema)

def test_load_and_prepare_data(spark, sample_data):
    """Prueba la función load_and_prepare_data."""
    delta_path = "/tmp/sample_delta"
    sample_data.write.format("delta").mode("overwrite").save(delta_path)

    feature_columns = ["transaction_velocity", "amount_velocity", "merchant_category_count", "hour_of_day", "day_of_week"]
    label_column = "is_fraud"

    df = load_and_prepare_data(spark, delta_path, feature_columns, label_column)
    assert "features" in df.columns
    assert label_column in df.columns

def test_add_class_weight(sample_data):
    """Prueba la función add_class_weight."""
    df = add_class_weight(sample_data, "is_fraud")
    assert "class_weight" in df.columns
    fraud_weights = df.filter(df.is_fraud == 1).select("class_weight").distinct().collect()
    non_fraud_weights = df.filter(df.is_fraud == 0).select("class_weight").distinct().collect()
    assert len(fraud_weights) == 1
    assert len(non_fraud_weights) == 1

def test_train_model_with_cv(sample_data):
    """Prueba la función train_model_with_cv."""
    experiment_name = "test_experiment"
    feature_columns = ["transaction_velocity", "amount_velocity", "merchant_category_count", "hour_of_day", "day_of_week"]
    assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
    sample_data = assembler.transform(sample_data)

    df_with_weights = add_class_weight(sample_data, "is_fraud")
    best_model = train_model_with_cv(df_with_weights, experiment_name)
    assert isinstance(best_model, LogisticRegressionModel)

def test_evaluate_model(spark, sample_data):
    """Prueba la función evaluate_model."""
    feature_columns = ["transaction_velocity", "amount_velocity", "merchant_category_count", "hour_of_day", "day_of_week"]
    assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
    df = assembler.transform(sample_data)

    lr = LogisticRegression(featuresCol="features", labelCol="is_fraud", weightCol="class_weight", maxIter=10)
    df_with_weights = add_class_weight(df, "is_fraud")
    model = lr.fit(df_with_weights)

    auc = evaluate_model(model, df_with_weights)
    assert isinstance(auc, float)
    assert 0.0 <= auc <= 1.0